In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold

from sklearn.preprocessing import OneHotEncoder,LabelEncoder

from xgboost import plot_importance
from xgboost import cv
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

from bayes_opt import BayesianOptimization
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

import matplotlib.pyplot as plt
from xgboost import plot_importance
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split
import warnings
from catboost import CatBoostRegressor
warnings.filterwarnings('ignore')

# **Data**

In [ ]:
train = pd.read_csv('/kaggle/input/umojahack-2023-africa-carbon-dioxide/Train(2).csv')
test = pd.read_csv('/kaggle/input/umojahack-2023-africa-carbon-dioxide/Test(3).csv')
ID = test.ID_LAT_LON_YEAR_WEEK

# **previous process**

In [ ]:
# missing value droped
col = ['UvAerosolLayerHeight_solar_azimuth_angle','UvAerosolLayerHeight_solar_zenith_angle',
       'UvAerosolLayerHeight_aerosol_pressure','UvAerosolLayerHeight_aerosol_optical_depth',
       'UvAerosolLayerHeight_sensor_zenith_angle','UvAerosolLayerHeight_sensor_azimuth_angle',
       'UvAerosolLayerHeight_aerosol_height'
      ]

train.drop(col,axis=1,inplace=True)
test.drop(col,axis=1,inplace=True)
# add Coordination place
train['Place'] = train['latitude'].astype(str) +'_' +train['longitude'].astype(str)
test['Place']  = test['latitude'].astype(str) +'_' +test['longitude'].astype(str)
train.drop(['latitude','longitude'],axis=1,inplace=True)
test.drop(['latitude','longitude'],axis=1,inplace=True)

# Date Added
for df in [train,test]:
    we = df.year*1000 + df.week_no* 10 + 1
    df['date'] = pd.to_datetime(we, format='%Y%W%w')
    df.drop('week_no',axis=1,inplace=True)
#     df.sort_values(by='date',inplace=True)
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['week_year'] = df['date'].dt.week.astype(str) +'-' +df['date'].dt.year.astype(str)
    df['month_year'] = df['date'].dt.month.astype(str) +'-' +df['date'].dt.year.astype(str)
    
    
# Date Defference
# Add difference between date by place
for df in [train,test]:
    df['date_diff'] = df['date'] - df.groupby('Place')['date'].transform(lambda x: x.min())
    df['date_diff'] = df['date_diff'].dt.days

# **Prerocessing**

In [ ]:
def Prerocessing(train,test):
    # Agregation
    feature = ['NitrogenDioxide_NO2_column_number_density','Cloud_surface_albedo','NitrogenDioxide_tropospheric_NO2_column_number_density','SulphurDioxide_SO2_column_number_density_15km','UvAerosolIndex_sensor_altitude','Ozone_O3_slant_column_number_density','NitrogenDioxide_absorbing_aerosol_index','Formaldehyde_HCHO_slant_column_number_density','CarbonMonoxide_cloud_height','Ozone_O3_column_number_density_amf','NitrogenDioxide_tropopause_pressure','NitrogenDioxide_cloud_fraction','NitrogenDioxide_sensor_azimuth_angle','Ozone_O3_column_number_density','CarbonMonoxide_sensor_altitude','CarbonMonoxide_solar_azimuth_angle','Cloud_cloud_fraction', 'SulphurDioxide_SO2_column_number_density', 'UvAerosolIndex_solar_azimuth_angle', 'SulphurDioxide_SO2_slant_column_number_density', 'SulphurDioxide_sensor_azimuth_angle', 'UvAerosolIndex_absorbing_aerosol_index', 'SulphurDioxide_cloud_fraction', 'Ozone_cloud_fraction', 'Cloud_solar_azimuth_angle', 'Formaldehyde_cloud_fraction', 'SulphurDioxide_sensor_zenith_angle', 'Ozone_sensor_azimuth_angle', 'Cloud_sensor_zenith_angle', 'SulphurDioxide_SO2_column_number_density_amf', 'NitrogenDioxide_sensor_zenith_angle', 'NitrogenDioxide_stratospheric_NO2_column_number_density', 'CarbonMonoxide_H2O_column_number_density', 'Cloud_cloud_base_pressure', 'NitrogenDioxide_sensor_altitude', 'Cloud_solar_zenith_angle', 'Ozone_O3_effective_temperature', 'Formaldehyde_tropospheric_HCHO_column_number_density', 'CarbonMonoxide_sensor_zenith_angle', 'Cloud_cloud_base_height', 'Cloud_cloud_optical_depth', 'Formaldehyde_tropospheric_HCHO_column_number_density_amf', 'NitrogenDioxide_NO2_slant_column_number_density', 'Ozone_sensor_zenith_angle', 'SulphurDioxide_solar_azimuth_angle', 'Cloud_cloud_top_pressure', 'CarbonMonoxide_sensor_azimuth_angle', 'Formaldehyde_solar_azimuth_angle', 'UvAerosolIndex_sensor_azimuth_angle', 'Formaldehyde_sensor_azimuth_angle', 'Formaldehyde_sensor_zenith_angle', 'Formaldehyde_solar_zenith_angle', 'Ozone_solar_azimuth_angle', 'Ozone_solar_zenith_angle', 'UvAerosolIndex_solar_zenith_angle', 'UvAerosolIndex_sensor_zenith_angle', 'CarbonMonoxide_CO_column_number_density', 'NitrogenDioxide_solar_azimuth_angle']
    print('<<Preprocessing Data>>>')
    def agg(fet_groupby):
        for data in(train,test):
            for fet in feature:
                data[fet_groupby+'_'+fet+'_min'] = data.groupby(fet_groupby)[fet].transform(lambda x: x.min())
                data[fet_groupby+'_'+fet+'_max'] = data.groupby(fet_groupby)[fet].transform(lambda x: x.max())
                data[fet_groupby+'_'+fet+'_mean'] = data.groupby(fet_groupby)[fet].transform(lambda x: x.mean())
                data[fet_groupby+'_'+fet+'_std'] = data.groupby(fet_groupby)[fet].transform(lambda x: x.std())
                data[fet_groupby+'_'+fet+'_var'] = data.groupby(fet_groupby)[fet].transform(lambda x: x.var())
                data[fet_groupby+'_'+fet+'_range'] = data.groupby(fet_groupby)[fet].transform(lambda x: x.min() - x.max())
#                 data[fet_groupby+'_'+fet+'_sem'] = data.groupby(fet_groupby)[fet].transform(lambda x: x.sem())
#                 data[fet_groupby+'_'+fet+'_sum'] = data.groupby(fet_groupby)[fet].transform(lambda x: x.sum())
                data[fet_groupby+'_'+fet+'_skew'] = data.groupby(fet_groupby)[fet].transform(lambda x: x.skew())

    tshape= train.shape
    eshape = test.shape
    print('Aggregation data...')
    agg_fet = ['Place','month','year','week_year','month_year','date']
    for fet in agg_fet:
        agg(fet)
    # print('Aggregation Done!')
    print('(Before Aggregation: train and test shape is:)')
    print('(train shape)',tshape, '(test shape)',eshape)
    print('(After Aggregation: train and test shape is):')
    print('(train shape)',train.shape, '(test shape)',test.shape)

    # Add difference between date by place
    for df in [train,test]:
        df['date_diff'] = df['date'] - df.groupby('Place')['date'].transform(lambda x: x.min())
        df['date_diff'] = df['date_diff'].dt.days

    # Mean Encoding
    # define the column i will be used th mean for it
    print('Target (mean) Endocing Start..')
    mean_feature = ['year','month_year','week_year','date']
    for fet in mean_feature: 
        mean_enc = train.groupby(fet)['emission'].mean()
        train[fet+'_mean_target_encoding'] = train[fet].map(mean_enc)
        test[fet +'_mean_target_encoding'] = test[fet].map(mean_enc)
    print('(After Mean Encoding train and test shape is):')
    print('train shape',train.shape, 'test shape',test.shape)

    # Label Encoding
    feats = ['month_year','week_year']
    le = LabelEncoder()
    for feat in feats:
        for d in [train,test]:
            le.fit(d[feat])
            d[feat] = le.transform(d[feat])
    
    train.drop(['ID_LAT_LON_YEAR_WEEK','Place','date'],axis=1)
    test.drop(['ID_LAT_LON_YEAR_WEEK','Place','date'],axis=1,inplace=True)
    print('<<All Done Ok...>>>')
    return train,test


In [ ]:
train, test = Prerocessing(train,test)

In [ ]:
# train.to_csv('preprocess_train.csv',index=False)
# test.to_csv('preprocess_test.csv',index=False)

# **Hyper Parameter**

In [ ]:
X = train.drop(['emission','ID_LAT_LON_YEAR_WEEK','Place','date'],axis=1)
y = train.emission

In [ ]:
# params = {'max_depth': [3,6,10,15],
#    'learning_rate': [0.01, 0.05, 0.1],
#    'n_estimators': [100, 200, 300, 500, 1000],
#    'colsample_bytree': [0.3, 0.7,.09],
#     'subsample':[0.1,.02]
#          }
# xgbr = xgb.XGBRegressor(seed = 42,tree_method='gpu_hist')
# clf = GridSearchCV(estimator=xgbr, param_grid=params, scoring='neg_mean_squared_error', verbose=1000)
# clf.fit(X, y)
# print("Best parameters:", clf.best_params_)
# print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

# **Kaggle HyperParameter**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [ ]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model, train_X, train_y,
                                    scoring="neg_mean_squared_error",
                                    cv=kfolds))
    return rmse

In [ ]:
# space = {

#         'n_estimators':hp.choice('n_estimators', np.arange(400, 1000, 10, dtype=int)),

#         'gamma': hp.uniform ('gamma', 1,15),

#         'subsample':hp.quniform('subsample', 0.5, 0.9, 0.01),

#         'eta':hp.quniform('eta', 0.05, 0.5, 0.01),

#         'eval_metric': 'rmse',

#     }
# def score(params):

#     model = XGBRegressor(**params,tree_method='gpu_hist')

#     model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)],

#               verbose=False, early_stopping_rounds=10)

#     Y_pred = model.predict(X_test)

#     score = np.sqrt(mean_squared_error(y_test, Y_pred)) 
# #     score=np.sqrt(-cross_val_score(model,train_X,train_y,cv=kfolds,scoring='neg_mean_squared_error'))

#     print(score)

#     return {'loss': score, 'status': STATUS_OK}    

# def optimize(trials, space):

#     best = fmin(score, space, algo=tpe.suggest, max_evals=100)

#     return best


# trials = Trials()

# best_params = optimize(trials, space)


# # Return the best parameters

# space_eval(space, best_params)

In [ ]:
# {'eta': 0.19,
#  'eval_metric': 'rmse',
#  'gamma': 5.420256552806366,
#  'n_estimators': 970,
#  'subsample': 0.78}

# **2.Lightgbm tuning¶**


In [ ]:
space = {

        'n_estimators':hp.choice('n_estimators', np.arange(400, 1200, 10, dtype=int)),
    
#         'min_child_weight': hp.uniform ('min_child_weight', 1,30),
    
        'max_depth':hp.choice('max_depth', np.arange(5, 13, 1, dtype=int)),

        'subsample':hp.quniform('subsample', 0.3, 0.9, 0.01),

        'learning_rate':hp.quniform('learning_rate', 0.05, 0.5, 0.01),
        
        'bagging_fraction': hp.uniform('bagging_fraction',0.5, 1),

        'eval_metric': 'rmse',

    }

In [ ]:
def score(params):

    model = LGBMRegressor(**params,device='gpu')

    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)],

              verbose=False, early_stopping_rounds=10)

    Y_pred = model.predict(X_test)

    score = np.sqrt(mean_squared_error(y_test, Y_pred)) 
#     score=np.sqrt(-cross_val_score(model,train_X,train_y,cv=kfolds,scoring='neg_mean_squared_error'))

    print(score)

    return {'loss': score, 'status': STATUS_OK}    

def optimize(trials, space):

    best = fmin(score, space, algo=tpe.suggest, max_evals=100)

    return best


trials = Trials()

best_params = optimize(trials, space)


# Return the best parameters

space_eval(space, best_params)

In [ ]:
space = {

        'n_estimators':hp.choice('n_estimators', np.arange(400, 1000, 10, dtype=int)),
    
        'max_depth':hp.choice('max_depth', np.arange(5, 13, 1, dtype=int)),

        'subsample':hp.quniform('subsample', 0.3, 0.9, 0.01),

        'learning_rate':hp.quniform('learning_rate', 0.05, 0.5, 0.01),
    }

def score(params):

    model = GradientBoostingRegressor(**params)

    model.fit(X_train, y_train)

    Y_pred = model.predict(X_test)

    score = np.sqrt(mean_squared_error(y_test, Y_pred)) 
#     score=np.sqrt(-cross_val_score(model,train_X,train_y,cv=kfolds,scoring='neg_mean_squared_error'))

    print(score)

    return {'loss': score, 'status': STATUS_OK}    

def optimize(trials, space):

    best = fmin(score, space, algo=tpe.suggest, max_evals=100)

    return best


trials = Trials()

best_params = optimize(trials, space)


# Return the best parameters

space_eval(space, best_params)
